In [1]:
# Required packages (install if needed):
!pip install rebound reboundx astroquery astropy numpy pandas

import rebound
import reboundx
import numpy as np
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy.time import Time
import warnings
from astropy.utils.exceptions import ErfaWarning
warnings.simplefilter('ignore', ErfaWarning)

Defaulting to user installation because normal site-packages is not writeable


In [2]:
initdate = "2011-01-01 00:00:00"

In [3]:
# --- PARAMETERS ---
JD_start = 2455562.5
JD_ref = 2451545.0
days_per_year = 365.25
t0 = (JD_start - JD_ref) / days_per_year

# --- SETUP SIMULATION ---
sim = rebound.Simulation()
sim.units = ('AU', 'yr', 'Msun')
sim.integrator = "ias15"
#sim.ri_ias15.min_dt = 1e-8  # stabilizes long-term integration

sim.add("Sun", hash="sun", date=initdate)
for planet in ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]:
    sim.add(planet, date=initdate)
sim.add("Bennu", date=initdate)
sim.particles[-1].m = 3.6858598e-20  # Set Bennu's mass in Msun

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Mercury'... 
Found: Mercury Barycenter (199) (chosen from query 'Mercury')
Searching NASA Horizons for 'Venus'... 
Found: Venus Barycenter (299) (chosen from query 'Venus')
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
Searching NASA Horizons for 'Mars'... 
Found: Mars Barycenter (4) (chosen from query 'Mars')
Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Searching NASA Horizons for 'Saturn'... 
Found: Saturn Barycenter (6) (chosen from query 'Saturn')
Searching NASA Horizons for 'Uranus'... 
Found: Uranus Barycenter (7) (chosen from query 'Uranus')
Searching NASA Horizons for 'Neptune'... 
Found: Neptune Barycenter (8) (chosen from query 'Neptune')
Searching NASA Horizons for 'Bennu'... 
Found: 101955 Bennu (1999 RQ36) (2101955) 


/home/nabillbox/rbx_venv/lib/python3.9/site-packages/rebound/horizons.py:181: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


In [4]:
# --- FETCH BENNU'S STATE VECTOR ---
#obj = Horizons(id='101955', location='500@0', epochs=JD_start, id_type='smallbody')
#vec = obj.vectors()[0]

#AU_per_km = 1 / 149597870.7
#sec_per_yr = 365.25 * 86400

#vx_auyr = 1.537787896744310E-01 * AU_per_km * sec_per_yr
#vy_auyr = -2.582218644800113E+01 * AU_per_km * sec_per_yr
#vz_auyr = -2.727643810842073E+00 * AU_per_km * sec_per_yr

#sim.add(x=-1.787897783189468E+08 * AU_per_km,
#        y=-3.511310048144390E+07 * AU_per_km,
#        z=-3.041228507657278E+06 * AU_per_km,
#        vx=vx_auyr,
#        vy=vy_auyr,
#        vz=vz_auyr,
#        m=3.922e-20,
#        hash="Bennu")  # test particle

In [5]:
# Yarkovsky constants
c = 63241.077  # speed of light in AU/yr
stef_boltz = 8.96e-16 # Stefan-Boltzmann constant in Msun / (yr^3 K^4)
lstar = 2.7e-4  # solar luminosity (Energy / time) in (Msun * AU^2 / yr^2 )

# --- BENNU PHYSICAL PROPERTIES ---
density = 2010363.7867  # Msun/AU^3
radius = 1.644e-9  # 246 meters i n AU ✅
Gamma = 8.48e-10  # thermal inertia Msun / yr^(5/2)
rotation_period = 4.9019e-4  # years
albedo = 0.044  
emissivity = 0.95  
k = 0  

In [6]:
# --- YARKOVSKY FORCE via REBOUNDx ---
rebx = reboundx.Extras(sim)
yark = rebx.load_force("yarkovsky_effect")

# Set the effect properties on the effect object:
yark.params["ye_c"] = c
yark.params["ye_lstar"] = lstar
yark.params["ye_stef_boltz"] = stef_boltz

# Sets parameters for Bennu
asteroid = sim.particles[-1]
asteroid.r = radius
asteroid.params["ye_flag"] = 0  #setting this flag to 0 will give us the full version of the effect
asteroid.params["ye_body_density"] = density
asteroid.params["ye_albedo"] = albedo
asteroid.params["ye_emissivity"] = emissivity
asteroid.params["ye_k"] = k
asteroid.params["ye_thermal_inertia"] = Gamma
asteroid.params["ye_rotation_period"] = rotation_period
asteroid.params["ye_spin_axis_x"] = 1      #vec['-1.787897783189468E+08']
asteroid.params["ye_spin_axis_y"] = 0      #vec['-3.511310048144390E+07']
asteroid.params["ye_spin_axis_z"] = 0      #vec['-3.041228507657278E+06']

rebx.add_force(yark)

In [7]:
# --- INTEGRATION SETUP ---
#n_outputs = 0.1  # every 0.1 years
#tmax = 1000  # 1 Myr
#times = np.linspace(t0, t0 + tmax, n_outputs)

n_outputs = 10000                    
years_per_step = 100            # Every 100 years for 1000000 years
total_time = n_outputs * years_per_step
times = t0 + np.arange(n_outputs) * years_per_step

elements = []
#a_vals = []
#e_vals = []

In [8]:
def get_orbit(sim):
    sun = sim.particles[0]
    p = sim.particles[-1]  # Bennu

    # Compute orbit relative to the Sun
    orbit = rebound.Orbit(p.x - sun.x,
                          p.y - sun.y,
                          p.z - sun.z,
                          p.vx - sun.vx,
                          p.vy - sun.vy,
                          p.vz - sun.vz,
                          m=p.m,
                          primary=sun)

    return orbit

In [9]:
# Define different spin axis configurations to test
spin_axis_variants = [
    (1, 0, 0),  # obliquity 90°
    #(0, 1, 0),  # obliquity 90°
    #(0, 0, 1),  # obliquity 0°
    #(1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)),  #obliquity ~54.7° # 45° in all directions
    #(-1, 0, 0)  # obliquity 90°
    #(0, -1, 0), # obliquity 90°
    #(0, 0, -1), # obliquity 180°
    #(1/np.sqrt(3), 1/np.sqrt(3), -1/np.sqrt(3)),  # obliquity ~125.3°
]

for i, (sx, sy, sz) in enumerate(spin_axis_variants):
    # Normalize spin axis vector (optional but recommended)
    #norm = np.linalg.norm([sx, sy, sz])
    #sx, sy, sz = sx/norm, sy/norm, sz/norm

    #print(f"\n=== Running Variant {i} ===")
    #print(f"Spin axis = ({sx:.3f}, {sy:.3f}, {sz:.3f})")
    
    # Clone the simulation (independent copy)
    sim_variant = sim.copy()
    asteroid_variant = sim_variant.particles[-1]
    
    # Re-initialize force (MUST be done after copy)
    rebx_variant = reboundx.Extras(sim_variant)
    yark_variant = rebx_variant.load_force("yarkovsky_effect")
    

    # Set Yarkovsky force parameters
    yark_variant.params["ye_c"] = c
    yark_variant.params["ye_lstar"] = lstar
    yark_variant.params["ye_stef_boltz"] = stef_boltz

    # Set asteroid physical and spin parameters
    asteroid_variant.params["ye_flag"] = 0
    asteroid_variant.params["ye_body_density"] = density
    asteroid_variant.params["ye_albedo"] = albedo
    asteroid_variant.params["ye_emissivity"] = emissivity
    asteroid_variant.params["ye_k"] = k
    asteroid_variant.params["ye_thermal_inertia"] = Gamma
    asteroid_variant.params["ye_rotation_period"] = rotation_period
    asteroid_variant.params["ye_spin_axis_x"] = sx
    asteroid_variant.params["ye_spin_axis_y"] = sy
    asteroid_variant.params["ye_spin_axis_z"] = sz

    rebx_variant.add_force(yark_variant)
    
    variant_elements = []

    for t in times:
        sim_variant.integrate(t)
        orbit = sim_variant.particles[-1]
        jd = JD_ref + t * days_per_year
        calendar_date = Time(jd, format='jd').iso.split()[0]

        variant_elements.append([
            t, jd, calendar_date,
            orbit.a,
            orbit.e,
            np.degrees(orbit.inc),
            np.degrees(orbit.Omega),
            np.degrees(orbit.omega),
            np.degrees(orbit.M)
        ])

    # Save to separate CSV file
    filename = f"[pro_retro] coba aja bennu 2011_sx{sx:.2f}_sy{sy:.2f}_sz{sz:.2f}.csv"
    df_variant = pd.DataFrame(variant_elements, columns=[
        "Time (yr since J2000)", "Julian Date", "Calendar Date",
        "a (AU)", "e", "i (deg)", "Omega (deg)", "omega (deg)", "M (deg)"
    ])
    df_variant.to_csv(filename, index=False)
    print(f"Saved: {filename}")

/home/nabillbox/rbx_venv/lib/python3.9/site-packages/rebound/simulation.py:192: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


Saved: [pro_retro] coba aja bennu 2011_sx1.00_sy0.00_sz0.00.csv
